In [14]:
import pickle
from copy import deepcopy
import math
import pandas as pd

activities_to_classify = {
        'Get/replace items from refrigerator/cabinets/drawers': 0,
        'Peel a cucumber': 1,
        'Clear cutting board': 2,
        'Slice a cucumber': 3,
        'Peel a potato': 4,
        'Slice a potato': 5,
        'Slice bread': 6,
        'Spread almond butter on a bread slice': 7,
        'Spread jelly on a bread slice': 8,
        'Open/close a jar of almond butter': 9,
        'Pour water from a pitcher into a glass': 10,
        'Clean a plate with a sponge': 11,
        'Clean a plate with a towel': 12,
        'Clean a pan with a sponge': 13,
        'Clean a pan with a towel': 14,
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 15,
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 16,
        'Stack on table: 3 each large/small plates, bowls': 17,
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 18,
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 19,
}

FPS = 30
ACTION_LENGTH = 5

def take_S04_annotations_RGB(timestamps, emg_data, type):
    
    calibration_val = pd.read_pickle(timestamps)['start'].iloc[0]

    emg_data = pd.read_pickle(emg_data)
    
    emg_data = emg_data[emg_data['file'] == 'S04_1.pkl']
    
    emg_data = emg_data.rename(columns={'file': 'video_id', 'description': 'narration', 'description_class': 'verb_class'})
    emg_data['participant_id'] = 'S04'
    emg_data['video_id'] = 'S04_1'
    emg_data['start_frame'] = ((emg_data['start'] - calibration_val) * FPS).astype(int)
    emg_data['stop_frame'] = ((emg_data['stop'] - calibration_val) * FPS).astype(int)
    emg_data['verb'] = emg_data['narration']
    emg_data = emg_data.drop(['emg_data'], axis = 1)
    emg_data = emg_data[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
    
    emg_data.to_pickle(f'an_annotations_rgb/S04_trial_{type}.pkl')    


def create_multimodal_annotations(full_data, split, spectogram):
    
    full_data = pd.read_pickle(full_data)
    full_data = full_data[full_data['file'] == 'S04_1.pkl']
    full_data = full_data.rename(columns={'description_class': 'verb_class'})
    full_data['participant_id'] = 'S04'
    full_data['video_id'] = 'S04_1'
    
    create_emg_features(deepcopy(full_data), split, spectogram)
    
    final_annotations = full_data[
        ['uid', 'participant_id', 'video_id', 'description', 'verb_class']]
    
    final_annotations.to_pickle(f"an_multimodal_annotations/S04_{split}.pkl")   

def create_emg_features(full_data, split):
    #os.chdir('./aml23-ego/EMG/')

    full_data = pd.read_pickle(full_data)
    #full_data = full_data[full_data['file'] == 'S04_1.pkl']
    full_data = full_data.rename(columns={'emg_data': 'features_EMG'})

    emg_features = full_data[
        ['uid', 'features_EMG']]
    emg_features = emg_features.to_dict(orient='index')
    emg_features = {'features': list(emg_features.values())}

    print(emg_features['features'][0]['features_EMG'].shape)
    features_name = f'../saved_features_an_multimodal/features_emg_5s_S04_{split}.pkl'

    full_data = full_data.rename(columns={'description_class': 'verb_class'})
    features_annotations=full_data[['uid', 'description', 'verb_class']]
    annotations_name=f'../../an_multimodal_annotations/allData_{split}.pkl'
    '''     with open(features_name, 'wb') as f:
        pickle.dump(emg_features, f)
    '''
    with open(annotations_name, 'wb') as f:
        pickle.dump(features_annotations, f)
        
def create_emg_spec_features(full_data, split):
    #print(os.getcwd())
    full_data = pd.read_pickle(full_data)

    full_data = full_data.rename(columns={'spectrogram': 'features_EMG_spectrogram'})

    emg_features = full_data[
        ['uid', 'features_EMG_spectrogram']]
    emg_features= emg_features.to_dict(orient='index')
    emg_features = {'features': list(emg_features.values())}
    # Creazione del dizionario desiderato

    features_name = f'../saved_features_an_multimodal/features_emg_5s_spectrogram_allData_{split}.pkl'

    with open(features_name, 'wb') as f:
        pickle.dump(emg_features, f)


In [5]:
take_S04_annotations_RGB(timestamps='action-net/S04_1.pkl', emg_data='an_multimodal_annotations/emg_trial_test.pkl', type='test')
take_S04_annotations_RGB(timestamps='action-net/S04_1.pkl', emg_data='an_multimodal_annotations/emg_trial_train.pkl', type='train')

In [ ]:
create_multimodal_annotations(full_data="new_emg_data_train.pkl", split='train')
create_multimodal_annotations(full_data="new_emg_data_test.pkl", split='test')

In [15]:
create_emg_features(full_data="./emg_5s_train.pkl", split='train')
create_emg_features(full_data="./emg_5s_test.pkl", split='test')

9937
(50, 16)
1109
(50, 16)


In [108]:
create_emg_spec_features(full_data="../EMG/EMG_data/emg_spectrogram_5s_test.pkl",split='test')
create_emg_spec_features(full_data="../EMG/EMG_data/emg_spectrogram_5s_train.pkl", split='train')

test
train
